<a href="https://colab.research.google.com/github/goerlitz/nlp-classification/blob/main/notebooks/10kGNAD/colab/21c_10kGNAD_huggingface_basic_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Optimization with HuggingFace Transformers

Adapted from https://huggingface.co/docs/transformers/custom_datasets#sequence-classification-with-imdb-reviews

Things we need
* a tokenizer
* tokenized input data
* a pretrained model
* evaluation metrics
* training parameters
* a Trainer instance

Notes
* [class labels can be included in the model config](https://github.com/huggingface/transformers/pull/2945#issuecomment-781986506) (a bit hacky)
* [fp16 is disabled on tesla P100 GPU in pytorch](https://discuss.pytorch.org/t/cnn-fp16-slower-than-fp32-on-tesla-p100/12146)
* [comparison of GPUS (K80, T4, P100, V100)](https://www.kaggle.com/general/198232)
* [GPU benchmark, mixed precision](https://medium.com/the-artificial-impostor/mixed-precision-training-on-tesla-t4-and-p100-d82e5d3b987d)

## Prerequisites

In [1]:
checkpoint = "distilbert-base-german-cased"
# checkpoint = "deepset/gbert-base"
# checkpoint = "deepset/gelectra-base"
# checkpoint = "deepset/gelectra-large"

project_name = f'10kgnad_hf__{checkpoint.replace("/", "_")}'

### Connect Google Drive

Will be used to save results

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from pathlib import Path

# define model path
root_path = Path('/content/gdrive/My Drive/')
base_path = root_path / 'Colab Notebooks/nlp-classification/'
model_path = base_path / 'models'

## Check GPU

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jan 15 13:45:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install APEX

https://stackoverflow.com/questions/57284345/how-to-install-nvidia-apex-on-google-colab

In [5]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [6]:
%%time
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8835, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 8835 (delta 28), reused 47 (delta 23), pack-reused 8767
Receiving objects: 100% (8835/8835), 14.50 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (6009/6009), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist

In [5]:
import apex

### Install Packages

In [5]:
%%time
!pip install -q -U transformers datasets >/dev/null
!pip install -q -U optuna >/dev/null

# check installed version
!pip freeze | grep optuna        # optuna==2.10.0
!pip freeze | grep transformers  # transformers==4.15.0
!pip freeze | grep "torch "      # torch==1.10.0+cu111

optuna==2.10.0
transformers==4.15.0
torch @ https://download.pytorch.org/whl/cu111/torch-1.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
CPU times: user 73.7 ms, sys: 43.5 ms, total: 117 ms
Wall time: 9.2 s


In [6]:
from transformers import logging

# hide progress bar when downloading tokenizer and model (a workaround!)
logging.get_verbosity = lambda : logging.NOTSET

## Load Dataset

In [7]:
from datasets import load_dataset

base_url = "https://raw.githubusercontent.com/tblock/10kGNAD/master/{}.csv"
data_files = {x: base_url.format(x) for x in ["train", "test"]}
dataset = (load_dataset('csv',
                        data_files=data_files,
                        sep=";",
                        quotechar="'",
                        names=["label", "text"]).
           class_encode_column("label"))

label_names = dataset["train"].features["label"].names

Using custom data configuration default-0e1a53e9f937c1cf
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-59d70429eaf283cf.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-a2cd8f79c68f7005.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-5dec691e070de180.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-7ee1cd4bfba8d7d3.arrow


In [8]:
print(dataset)
print("labels:", label_names)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 9245
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1028
    })
})
labels: ['Etat', 'Inland', 'International', 'Kultur', 'Panorama', 'Sport', 'Web', 'Wirtschaft', 'Wissenschaft']


{'label': 5,
 'text': '21-Jähriger fällt wohl bis Saisonende aus. Wien – Rapid muss wohl bis Saisonende auf Offensivspieler Thomas Murg verzichten. Der im Winter aus Ried gekommene 21-Jährige erlitt beim 0:4-Heimdebakel gegen Admira Wacker Mödling am Samstag einen Teilriss des Innenbandes im linken Knie, wie eine Magnetresonanz-Untersuchung am Donnerstag ergab. Murg erhielt eine Schiene, muss aber nicht operiert werden. Dennoch steht ihm eine mehrwöchige Pause bevor.'}

## Data Preprocessing

* Loading the same Tokenizer that was used with the pretrained model.
* Define function to tokenize the text (with truncation to max input length of model.
* Run the tokenization

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess_function(examples):
    # return tokenizer(examples["text"], truncation=True, max_length=128)
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True).remove_columns("text")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-9978cdff5e07f8bd.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

### Use Dynamic Padding

Apply panding only on longest text in batch - this is more efficient than applying padding on the whole dataset.

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Define Evaluation Metrics

The funtion that computes the metrics needs to be passed to the Trainer.

In [11]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
import numpy as np
from typing import Dict

def compute_metrics(eval_preds):
    """The function that will be used to compute metrics at evaluation.
    Must take a :class:`~transformers.EvalPrediction` and return a dictionary
    string to metric values."""
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)
    return {
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='macro'),
        "precision": precision_score(labels, preds, average='macro'),
        "recall": recall_score(labels, preds, average='macro'),
        "mcc": matthews_corrcoef(labels, preds),
        }

## Hyperparameter Tuning

In [12]:
from transformers import TrainerCallback
from optuna.trial import Trial, TrialState
from optuna.study._study_direction import StudyDirection
import pandas as pd

# https://github.com/huggingface/transformers/blob/v4.14.1/src/transformers/trainer_callback.py#L505
# https://huggingface.co/docs/transformers/main_classes/callback#transformers.TrainerCallback

import logging
logging.getLogger(__name__).setLevel(logging.INFO)
log = logging.getLogger(__name__)

class TrialLogAndPruningCallback(TrainerCallback):
    """Stores eval metrics at each evaluation step in the trial user attrs."""
    def __init__(self, trial: Trial, objectives=None, warmup_steps=0, min_trials=7):
        self.study = trial.study
        self.trial = trial
        self.param_keys = ["num_train_epochs", "per_device_train_batch_size"]
        self.param_vals = [trial.params[k] for k in self.param_keys]

        log.warning(f"fixed params: {list(zip(self.param_keys, self.param_vals))}")

        if objectives == None:
            self.objectives = ["eval_loss"]
        else:
            self.objectives = objectives
        self._warmup_steps = warmup_steps
        self._min_trials = max(1, int(min_trials))

        log.warning(f"objectives: {self.objectives}, directions: {self.study.directions}, warmup={self._warmup_steps}, min_trials={self._min_trials}")
        log.warning(f"params: {trial.params}")
        

    def _filter_trials(self, complete_trials):
        """Select only trials with same parameter values"""
        # values = [self.trial.params[k] for k in keys]
        return [t for t in complete_trials if self.param_vals == [t.params[k] for k in self.param_keys]]

    def _prune(self, step: int, metrics) -> bool:
        """Median Pruning on multiple objectives."""
        if step < self._warmup_steps:
            # log.warning(f"less than warmup steps {step}<{self._warmup_steps}")
            return False

        # get all completed trials
        complete_trials = self.study.get_trials(deepcopy=False,
                                                states=[TrialState.COMPLETE])
        # only compare trials with same batch size and epochs
        complete_trials = self._filter_trials(complete_trials)
        n_trials = len(complete_trials)

        # check minimal number of trial required
        if n_trials < self._min_trials:
            # log.warning(f"less than min trials {n_trials}<{self._min_trials}")
            return False

        # log.warning(f"checking {step}: {metrics}")

        # sanity check
        has_metrics = [o in metrics.keys() for o in self.objectives]
        if not all(has_metrics):
            log.warning(f"missing objective metrics {list(zip(self.objectives, has_metrics))}")

        # extract metrics from trials
        # print(f"fetching metrics of {n_trials} complete trials")
        trial_metrics = []
        for t in complete_trials:
            # print(str(step), "in keys?", str(step) in t.user_attrs.keys(), t.user_attrs.keys())
            if str(step) in t.user_attrs.keys():
                trial_metrics.append(t.user_attrs[str(step)])
        n_metrics = len(trial_metrics)

        # compute median for each metric over all trials
        median = pd.DataFrame(trial_metrics).median()

        # log.warning(f"median of {n_metrics}/{n_trials}: {median.to_dict()}")

        # compare current metric value with median
        prune_state = []
        for i, o in enumerate(self.objectives):
            if self.study.directions[i] == StudyDirection.MAXIMIZE:
                prune_state.append(metrics[o] <= median[o])
            else:
                prune_state.append(metrics[o] > median[o])
        
        met = ",".join([f"{m}={metrics[m]:.4}/{median[m]:.4}" for m in self.objectives])
        print(f"prune? step={step}, warmup={self._warmup_steps}, complete_trials={n_trials}, metrics={n_metrics} -> {met}; {prune_state}")

        # all metrics must be marked for pruning
        return all(prune_state)
    
    def on_evaluate(self, args, state, control, lr_scheduler, metrics, **kwargs):
        step = state.global_step
        values = {**metrics, "lr": lr_scheduler.get_last_lr()[-1]}
        self.trial.set_user_attr(str(step), values)

        # pruning
        if self._prune(step, metrics):
            print(f"pruning trial at step {step}")
            # control.should_training_stop = True  # not needed
            raise optuna.TrialPruned()

In [16]:
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import shutil

def hp_space(trial: Trial):
    """A function that defines the hyperparameter search space.
    To be used in :obj:`Trainer.hyperparameter_search`."""
    return {
        # "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True),  # distilbert/bert
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 4e-4, log=True),  # distilbert 1 epoch
        # "learning_rate": trial.suggest_float("learning_rate", 6e-5, 2e-4, log=True),  # electra
        # "num_train_epochs": trial.suggest_categorical("num_train_epochs", [1]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8,16,32]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-3, 1e-2, log=True),
        # "weight_decay": trial.suggest_categorical("weight_decay", [1e-3, 0.0]),
    }

best_model_dir = "best_model_trainer"

def best_model_callback(study, trial):
    """Save the model from a best trial"""
    for t in study.best_trials:
        if t.number == trial.number:
            print("This is a new besttrial", trial.number)
        
            out_filename = model_path / f"{project_name}_t{trial.number}"
            shutil.make_archive(out_filename, 'zip', f"{project_name}/{best_model_dir}")

def model_init(trial: Trial):
    """A function that instantiates the model to be used."""

    # We want to include the label names and save them together with the model.
    # The only way to do this is to create a Config and put them in. 
    config = AutoConfig.from_pretrained(
            checkpoint,
            num_labels=len(label_names),
            id2label={i: label for i, label in enumerate(label_names)},
            label2id={label: i for i, label in enumerate(label_names)},
            )

    return AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

def objective(trial: Trial):

    # get hyperparameters choice
    hp = hp_space(trial)
    lr = hp["learning_rate"]
    bs = hp["per_device_train_batch_size"]
    epochs = hp["num_train_epochs"]
    weight_decay = hp["weight_decay"]
    # label_smoothing_factor = hp["label_smoothing_factor"]

    eval_rounds_per_epoch = 5
    eval_steps = dataset["train"].num_rows / bs // eval_rounds_per_epoch

    training_args = TrainingArguments(
        output_dir=str(project_name),
        report_to=[],
        log_level="error",
        disable_tqdm=False,

        evaluation_strategy="steps",
        eval_steps=eval_steps,
        logging_steps=eval_steps,
        save_strategy="steps",
        save_steps=eval_steps,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        # hyperparameters
        num_train_epochs=epochs,
        learning_rate=lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        weight_decay=weight_decay,
        # label_smoothing_factor=label_smoothing_factor,

        fp16=True,  # fp16 needs apex. but disabled on Tesla P100 by pytorch
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[TrialLogAndPruningCallback(trial, objectives=["eval_loss", "eval_f1"], min_trials=700, warmup_steps=eval_steps*3)]
        # callbacks=[TrialPruningCallback(trial)]
    )

    # train model and save best model from evaluations
    # needs 'load_best_model_at_end=True'
    trainer.train()
    trainer.save_model(f"{project_name}/{best_model_dir}")

    result = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

    # store eval metrics in trial
    trial.set_user_attr("eval_result", result)
    
    # return result["eval_loss"]
    return result["eval_loss"], result["eval_f1"]

## Hyperparameter Tuning

In [17]:
# run_study(
#     dataset=dataset,
#     max_seq_length=None,
#     checkpoint="distilbert-base-german-cased",
#     batch_size=32,
#     train_epochs=[2,3]
#     learning_rate=MinMax(4e-5, 4e-4, log=True),
#     weight_decay=MinMax(1e-3, 1e-2, log=True),
#     eval_metrics={"loss":"minimize", "f1":"maximize"}
# )

In [ ]:
import optuna
from optuna.storages import RDBStorage

db_path = "/content/gdrive/My Drive/Colab Notebooks/nlp-classification/"
db_name = "10kgnad_optuna"
# study_name = checkpoint + "_multi_epoch234"
# study_name = checkpoint + "_loss-f1_bs32_epoch23"
study_name = checkpoint + "_loss-f1_bs8-16-32_ep2"

# automatically change the state of a stale trial to TrialState.FAIL from TrialState.RUNNING
storage = RDBStorage(url=f"sqlite:///{db_path}{db_name}.db", heartbeat_interval=60, grace_period=120)

# https://stackoverflow.com/questions/59129812/how-to-avoid-cuda-out-of-memory-in-pytorch
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

# multi objective study
# https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/002_multi_objective.html#sphx-glr-tutorial-20-recipes-002-multi-objective-py
study = optuna.create_study(study_name=study_name,
                            directions=["minimize", "maximize"],
                            # storage=f"sqlite:///{db_path}{db_name}.db",
                            storage=storage,
                            load_if_exists=True,)

# give some hyperparameters that are presumably good
# study.enqueue_trial(
#     {
#         "learning_rate": 8e-5,
#         "weight_decay": 1e-3,
#     }
# )


study.optimize(objective, n_trials=200, callbacks=[best_model_callback])

# study.best_params

[I 2022-01-15 15:07:38,250] Using an existing study with name 'distilbert-base-german-cased_loss-f1_bs8-16-32_ep2' instead of creating a new one.
fixed params: [('num_train_epochs', 2), ('per_device_train_batch_size', 16)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=345.0, min_trials=700
params: {'learning_rate': 1.5841336575732623e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 16, 'weight_decay': 0.0012759610636928096}


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
115,1.542100,1.002562,0.740272,0.703826,0.796412,0.677431,0.704291
230,0.855500,0.662495,0.825875,0.823235,0.846766,0.808833,0.800715
345,0.630300,0.568661,0.831712,0.830955,0.837120,0.837478,0.810011
460,0.563400,0.484695,0.852140,0.851814,0.850074,0.856939,0.830884
575,0.520200,0.448346,0.860895,0.864824,0.864596,0.867329,0.840889
690,0.460600,0.441752,0.857977,0.861155,0.869312,0.855649,0.837579
805,0.455200,0.421880,0.868677,0.865653,0.866656,0.865927,0.849601
920,0.388300,0.407480,0.871595,0.873175,0.877211,0.870309,0.852883
1035,0.378100,0.416235,0.871595,0.871517,0.872104,0.873035,0.853223
1150,0.360900,0.403248,0.878405,0.878800,0.879782,0.878504,0.860689


[I 2022-01-15 15:15:46,398] Trial 2 finished with values: [0.40324777364730835, 0.8788004284023483] and parameters: {'learning_rate': 1.5841336575732623e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 16, 'weight_decay': 0.0012759610636928096}. 
fixed params: [('num_train_epochs', 2), ('per_device_train_batch_size', 32)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=171.0, min_trials=700
params: {'learning_rate': 0.00013921866503418376, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.006937397778793355}


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
57,0.944100,0.592739,0.827821,0.822868,0.850284,0.811279,0.803951
114,0.658900,0.508770,0.841440,0.841007,0.855554,0.833603,0.819027
171,0.494500,0.442673,0.857004,0.851598,0.855903,0.854771,0.837236
228,0.488300,0.437384,0.850195,0.850337,0.848005,0.856762,0.828839
285,0.427700,0.417069,0.859922,0.861117,0.860701,0.865723,0.840363
342,0.294000,0.381199,0.889105,0.886385,0.890807,0.882892,0.872960
399,0.288300,0.369089,0.893969,0.890636,0.895015,0.888161,0.878585
456,0.265200,0.371665,0.888132,0.888563,0.891101,0.886935,0.871874
513,0.203100,0.387259,0.887160,0.886977,0.886576,0.890186,0.871255
570,0.234100,0.352662,0.894942,0.892860,0.889610,0.896527,0.879753


[I 2022-01-15 15:23:36,411] Trial 3 finished with values: [0.35266202688217163, 0.8928603247918959] and parameters: {'learning_rate': 0.00013921866503418376, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.006937397778793355}. 


...

Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
57,0.940000,0.730445,0.794747,0.783917,0.841615,0.757233,0.769138
114,0.749700,0.606436,0.811284,0.812678,0.813355,0.818257,0.784986
171,0.617600,0.715555,0.756809,0.745780,0.787661,0.750381,0.731361
228,0.571000,0.479522,0.849222,0.850794,0.854997,0.854110,0.828456
285,0.510200,0.466681,0.835603,0.834413,0.832388,0.838896,0.812083
342,0.336100,0.450744,0.861868,0.859386,0.866455,0.855456,0.842024
399,0.308300,0.405104,0.867704,0.869015,0.874470,0.865403,0.848584
456,0.267200,0.414945,0.882296,0.881000,0.882005,0.882260,0.865542
513,0.239300,0.395568,0.874514,0.876370,0.875602,0.879081,0.856650
570,0.243700,0.373773,0.890078,0.891184,0.892715,0.890594,0.874195


[I 2022-01-15 22:14:14,265] Trial 52 finished with values: [0.37377315759658813, 0.8911836868441486] and parameters: {'learning_rate': 0.0003355274672670145, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.002987271061042641}. 
fixed params: [('num_train_epochs', 2), ('per_device_train_batch_size', 32)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=171.0, min_trials=700
params: {'learning_rate': 3.3780419094054394e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.00199180792849344}


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
57,1.467500,0.856826,0.790856,0.774230,0.835616,0.749999,0.760807
114,0.793100,0.597858,0.828794,0.822912,0.845480,0.811767,0.804078
171,0.567700,0.509920,0.843385,0.839913,0.848734,0.841814,0.822363
228,0.510400,0.456393,0.856031,0.854594,0.861941,0.854534,0.835529
285,0.462500,0.428853,0.862840,0.865798,0.862796,0.871819,0.843414
342,0.382700,0.413390,0.868677,0.868470,0.877193,0.862588,0.849686
399,0.392900,0.401583,0.864786,0.862953,0.867315,0.861320,0.845302
456,0.338500,0.376990,0.877432,0.874833,0.877667,0.873206,0.859561
513,0.302100,0.397914,0.871595,0.869567,0.871408,0.872312,0.853662
570,0.320600,0.371048,0.881323,0.878747,0.879879,0.878437,0.864028


[I 2022-01-15 22:22:05,779] Trial 53 finished with values: [0.3710477948188782, 0.8787473073057632] and parameters: {'learning_rate': 3.3780419094054394e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.00199180792849344}. 
fixed params: [('num_train_epochs', 2), ('per_device_train_batch_size', 16)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=345.0, min_trials=700
params: {'learning_rate': 0.00012952108947609435, 'num_train_epochs': 2, 'per_device_train_batch_size': 16, 'weight_decay': 0.004050497059826461}


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
115,0.891600,0.694186,0.798638,0.791896,0.819441,0.786111,0.771607
230,0.673300,0.589224,0.824903,0.815928,0.834841,0.806692,0.799738
345,0.532500,0.562210,0.824903,0.816604,0.841623,0.814764,0.802841
460,0.511400,0.428867,0.862840,0.862604,0.864090,0.865869,0.843232
575,0.464600,0.442442,0.848249,0.849544,0.851079,0.857312,0.827937
690,0.324500,0.431394,0.881323,0.879675,0.878921,0.882416,0.864421
805,0.307000,0.418317,0.870623,0.854641,0.865376,0.853774,0.852008
920,0.249200,0.411139,0.885214,0.883369,0.885369,0.883398,0.868838


In [ ]:
!ls -lahtr $project_name

## Hyperparameter Tuning

https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.hyperparameter_search

In [ ]:
# disable transformer warnings like "Some weights of the model checkpoint ..."
logging.set_verbosity_error()


training_args = TrainingArguments(
    output_dir=str(project_name),
    report_to=[],
    log_level="error",
    disable_tqdm=False,

    evaluation_strategy="steps",
    # eval_steps=eval_steps,
    save_strategy="steps",
    # save_steps=eval_steps,
    # load_best_model_at_end=False,
    # metric_for_best_model="eval_loss",
    # greater_is_better=False,
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
# best = trainer.hyperparameter_search(
#     hp_space=hp_space,
#     compute_objective=objective,
#     n_trials=2
# )